In [2]:
pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.comNote: you may need to restart the kernel to use updated packages.



# 1.Problem description, assumptions
-------------------------------
In RMS, we have a set of i operations and m set of machines for processing. Based on the processing route, jobs are divided into k sets. Each Job in a group has the same processing route. Machines are reconfigured To prepare machines for each group; therefore, we need k different configurations. Since each configuration's jobs have the same processing route, each configuration is a flow shop problem. It is required to set up the system for a new configuration. This setup's size depends on the two following configurations (going from configuration to which configurations). Reconfiguring from a configuration to an entirely different one is more extensive. Hence, the configuration sequence is also a single machine problem with sequence-dependent setup times. All in all, the RMS problem is a combined problem of (permutation flow shop and single machine problems).
This research discusses the design of RCMSs. These systems are defined as multi RMCs, RMTs, material handling, and storehouse systems. Each RMT within an RMC has primary and configuration modules. A particular mixture of both different primary and configuration modules gives a unique operational capability to the RMT.
The CRMS design problem involves the grouping of RMTs in each RMC by using information about the operation sequence for each part type and the assignment of each auxiliary module to the RMTs. In this research, an optimization model for the design of CRMS is proposed. The goal of the defined model is to best balance sustainability, social, environmental, and economical.
The proposed optimization model adopts the following assumptions according to the standard literature within CRMS modeling:
Operation-based process plan for the parts is known and fixed; Requirement of modules and machine-module compatibility information is known and unchangeable;
Size limit for each RMC is to be given;
Unique assignment of the machines to each cell;
Auxiliary modules are available when needed;
CNC machines are not considered in the optimization at this stage since they do not include elements of configurability.
These assumptions are still realistic and representative of typical production environments. Furthermore, the model is flexible and can be adapted to match different beliefs (e.g., mono or multi-product model, mono or multi-period model).


# Inserting Library

In [3]:
import gurobipy as gp
from gurobipy import GRB
from random import randint

# 2.Define Sets(Indices)

$ i\,\,\,\,\,\,\,\,\,\,\,{\rm{Products}}\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,i\,\,\, = \,\,1,\,...\,,P $

In [ ]:
num_Products = 3 # set of Products
products = [f"pro{i}" for i in range(num_Products)]

$ o\,\,\,\,\,\,\,\,\,\,{\rm{Operations}}\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,o\,\, = \,\,1,\,..\,.\,,O $

In [ ]:
num_operations = 5 # set of operations
operations = [f"opr{o}" for o in range(num_operations)]

$ m\,\,\,\,\,\,\,\,\,\begin{array}{*{20}{l}}
{{\rm{Machines}}}
\end{array}\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,m\, = \,1,...,M $

In [ ]:
num_machines = 5 # set of machines
machines = [f"mac{m}" for m in range(num_machines)]

$ k\,\,\,\,\,\,\,\,\,\,{\rm{Configurations}}\,\,\,\,\,\,\,k\, = \,\,1,...,K{\rm{\;}}\, $

In [ ]:
num_configurations = 5 # set of configurations
configurations = [f"con{k}" for k in range(num_configurations)]

$ j\,\,\,\,\,\,\,\,\,\,{\rm{Machine \, Cells}}\,\,\,\,\,\,\,j\, = \,\,1,...,J{\rm{\;}}\, $

In [ ]:
num_cells = 2 # set of cells
cells = [f"cell{c}" for c in range(num_cells)]

$ t\,\,\,\,\,\,\,\,\,\,\,\begin{array}{*{20}{l}} {{\rm{Time \, Periods}}} \end{array}\,\,\,\,\,\,\,\,\,\,t\,\, = 1,...,T $

In [ ]:
num_times = 6 # set of cell
times = [f"time{c}" for c in range(num_times)]

# 3.Define Parameters

$ {P_j}\,\,\,\,\,\,\,\,\,\,\,\,\,\,{\rm{Maximum \, number \, of \, machines \, in \, each \, cell}}\,j $

| cell number  | cell1   | cell2   |
|--------------|---------|---------|
| capacity     |     3   |    3    |

In [ ]:
max_machines_in_cell = 3 #Pj
pj = {c : max_machines_in_cell for c in cells}

${G_{omk}}\,\,\,\,\,\,\,\,\,\,{\rm{1 \, if \, operation \,}}\,o\,{\rm{can \, be \, performed \, on \, machine}}\,m\,{\rm{using \, configurations\, module\, type\,}}\,k\,{\rm{; }}0{\rm{\, otherwise\,}}\,\left( {\,Binary\,} \,\right)$

In [ ]:
gomk ={(o,m,k): 0 for o in operations for m in machines for k in configurations }
gomk_update = ({
   ('opr0','mac0','con0') : 1, 
   ('opr0','mac1','con3') : 1,
   ('opr0','mac2','con4') : 1,
   
   ('opr1','mac0','con1') : 1,
   ('opr1','mac1','con3') : 1,
   ('opr1','mac1','con1') : 1,
   
   ('opr2','mac2','con0') : 1,
   ('opr2','mac4','con2') : 1,
   ('opr2','mac3','con1') : 1,
   
   ('opr3','mac1','con4') : 1,
   ('opr3','mac3','con4') : 1,
   ('opr3','mac4','con0') : 1,
   
   ('opr4','mac0','con3') : 1,
   ('opr4','mac2','con2') : 1,
   ('opr4','mac3','con0') : 1,
   

   
    })
gomk.update(gomk_update)

${t_{ijj'}}\,\,\,\,\,\,\,\,\,\,\,\,\,{\rm{Travel\, time\, for\, product}}\,i\,{\rm{\,from\, cell\,}}\,{\rm{j}}\,{\rm{\,to\, cell\,}}\,\,j'\,(\min )$

In [ ]:
tijj1 ={(i,j,j1): 0 for i in products for j in cells for j1 in cells }
tijj1_update = {
    ('pro0','cell0','cell1') : 0.5,
    ('pro0','cell0','cell2') : 0.75,
    ('pro0','cell1','cell2') : 1,
    
    ('pro1','cell0','cell1') : 1,
    ('pro1','cell0','cell2') : 1.5,
    ('pro1','cell1','cell2') : 2,
    
    ('pro2','cell0','cell1') : 1,
    ('pro2','cell0','cell2') : 1.25,
    ('pro2','cell1','cell2') : 1.5    
    
    }
tijj1.update(tijj1_update)

${\lambda _{mk}}\,\,\,\,\,\,\,\,\,\,\,\,\begin{array}{*{20}{l}}
{{\rm{Assembly\, time\, of\, }}\,{\rm{\,configurations\,}}\,k\,{\rm{\,on\, machine}}\,m\,}
\end{array}(\min )$

In [ ]:
lamk ={(m,k): 0 for m in machines for k in configurations}
listlamk = {
        ('mac0','con0') : 4,
        ('mac0','con1') : 3,
        ('mac0','con3') : 5,
        
        ('mac1','con3') : 6,
        ('mac1','con4') : 2,

        
        ('mac2','con0') : 3,
        ('mac2','con4') : 4,
        ('mac2','con2') : 5,
        
        ('mac3','con4') : 2,
        ('mac3','con1') : 4,
        ('mac3','con0') : 6,
        
        ('mac4','con0') : 4,
        ('mac4','con2') : 3

        
        
        }

for i in listlamk:
    listlamk[i]= listlamk[i]*6
lamk.update(listlamk)